In [5]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

from typing import List, Dict, Tuple, Set, Optional
from dataclasses import dataclass
from collections import defaultdict

## Create Synthetic Loan data

In [3]:
# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Define constants
INVESTORS = ["BankOfAmerica", "JPMorgan", "WellsFargo", "CitiBank", "GoldmanSachs", "MorganStanley"]
NOTE_RATES = [5.5, 5.75, 6.0, 6.25, 6.5, 6.75, 7.0, 7.25, 7.5]
SECURITY_TYPES = ["GNMA II", "CONV"]

def generate_loan_data(num_loans=1000, output_file="loans.csv"):
    """
    Generate synthetic loan data with stochastic distribution
    
    Args:
        num_loans: Number of loans to generate
        output_file: Path to save the CSV file
    """
    loans = []
    
    # Generate loans with randomized attributes
    for i in range(1, num_loans + 1):
        loan_number = f"LOAN-{i:05d}"
        note_rate = random.choice(NOTE_RATES)
        note_type = random.choice(SECURITY_TYPES)
        
        # Loan amount between $80k and $800k
        loan_amount = round(random.uniform(80000, 800000), 2)
        
        # Assign to random investor
        investor = random.choice(INVESTORS)
        
        loans.append({
            "loan_number": loan_number,
            "note_rate": note_rate,
            "note_type": note_type,
            "loan_amount": loan_amount,
            "investor": investor
        })
    
    # Convert to DataFrame and save
    loans_df = pd.DataFrame(loans)
    loans_df.to_csv(output_file, index=False)
    
    # Calculate statistics for loan pools
    loan_pools = loans_df.groupby(['investor', 'note_rate', 'note_type']).agg(
        loan_count=('loan_number', 'count'),
        total_amount=('loan_amount', 'sum')
    ).reset_index()
    
    print(f"Generated {len(loans_df)} loans")
    print(f"Total loan amount: ${loans_df['loan_amount'].sum():,.2f}")
    print(f"Created {len(loan_pools)} unique loan pools")
    
    # Calculate investor totals
    investor_totals = loans_df.groupby('investor')['loan_amount'].sum().sort_values(ascending=False)
    print("\nLoan amounts by investor:")
    for investor, total in investor_totals.items():
        print(f"  {investor}: ${total:,.2f}")
    
    # Calculate loan pools by type
    type_totals = loans_df.groupby('note_type')['loan_amount'].sum()
    print("\nLoan amounts by type:")
    for note_type, total in type_totals.items():
        print(f"  {note_type}: ${total:,.2f}")
    
    # Show largest loan pools
    print("\nTop 10 loan pools by amount:")
    top_pools = loan_pools.sort_values('total_amount', ascending=False).head(10)
    for _, pool in top_pools.iterrows():
        print(f"  {pool['investor']} - {pool['note_type']} at {pool['note_rate']}%: ${pool['total_amount']:,.2f} ({pool['loan_count']} loans)")
    
    return loans_df

In [4]:
generate_loan_data(1000, "loans.csv")

Generated 1000 loans
Total loan amount: $441,229,419.15
Created 108 unique loan pools

Loan amounts by investor:
  CitiBank: $83,711,517.30
  JPMorgan: $78,294,475.39
  MorganStanley: $78,156,861.02
  GoldmanSachs: $70,860,777.25
  WellsFargo: $66,021,613.24
  BankOfAmerica: $64,184,174.95

Loan amounts by type:
  CONV: $219,656,103.24
  GNMA II: $221,573,315.91

Top 10 loan pools by amount:
  JPMorgan - CONV at 7.5%: $8,184,644.57 (16 loans)
  MorganStanley - CONV at 5.75%: $7,652,689.26 (15 loans)
  CitiBank - CONV at 7.0%: $7,254,742.08 (14 loans)
  JPMorgan - GNMA II at 6.25%: $7,122,848.36 (12 loans)
  MorganStanley - CONV at 6.75%: $6,824,005.07 (14 loans)
  CitiBank - GNMA II at 6.0%: $6,526,562.27 (15 loans)
  CitiBank - CONV at 6.0%: $6,416,199.95 (12 loans)
  CitiBank - GNMA II at 5.5%: $6,156,541.05 (13 loans)
  JPMorgan - CONV at 5.5%: $6,032,816.08 (10 loans)
  MorganStanley - GNMA II at 5.75%: $5,930,496.03 (12 loans)


,loan_number,note_rate,note_type,loan_amount,investor
0,LOAN-00001,5.75,GNMA II,613916.36,JPMorgan
1,LOAN-00002,6.25,GNMA II,610259.27,MorganStanley
2,LOAN-00003,7.50,GNMA II,505154.61,BankOfAmerica
3,LOAN-00004,5.50,GNMA II,237419.34,GoldmanSachs
4,LOAN-00005,5.50,GNMA II,595534.12,MorganStanley
...,...,...,...,...,...
995,LOAN-00996,6.00,CONV,181123.41,MorganStanley
996,LOAN-00997,7.00,GNMA II,263088.30,JPMorgan
997,LOAN-00998,6.25,GNMA II,749755.71,CitiBank
998,LOAN-00999,6.25,CONV,673678.98,JPMorgan


# The Algorithm

## Create Class objects for performing the algorithm on

In [6]:
@dataclass
class MBS:
    """Class representing a Mortgage-Backed Security"""
    issuer: str
    expiration_date: datetime
    coupon: float
    type: str
    amount: int  # Amount in $100k increments (e.g., 5 = $500k)
    id: str = None  # Composite ID for tracking

    def __post_init__(self):
        if not self.id:
            self.id = f"{self.issuer}_{self.expiration_date.strftime('%Y-%m-%d')}_{self.coupon}_{self.type}"

@dataclass
class Investor:
    """Class representing an Investor with their constraints"""
    name: str
    allowed_issuers: List[str]
    min_expiration_date: datetime
    min_allocation: int  # Minimum allocation in $100k units

@dataclass
class LoanPool:
    """Represents a pool of loans with the same characteristics"""
    investor: str
    note_rate: float
    note_type: str
    total_amount: int  # Total amount in $100k units
    
    @property
    def key(self) -> str:
        """Generate a unique key for this loan pool"""
        return f"{self.investor}_{self.note_rate}_{self.note_type}"

@dataclass
class Allocation:
    """Result of MBS allocation"""
    loan_pools_allocation: Dict[str, int]  # Pool key -> amount allocated
    mbs_allocation: Dict[str, Dict[str, int]]  # MBS ID -> {pool key -> amount}
    total_allocated: int = 0
    
    def __post_init__(self):
        # Calculate total allocated if not provided
        if self.total_allocated == 0:
            for mbs_id, pool_allocations in self.mbs_allocation.items():
                for pool_key, amount in pool_allocations.items():
                    self.total_allocated += amount

## Define functions

In [7]:
def load_data(mbs_file: str, issuers_file: str, loans_file: str) -> Tuple[Dict[str, MBS], Dict[str, Investor], Dict[str, LoanPool]]:
    """
    Load data from CSV files and return as dictionaries for efficient access
    """
    # Load MBS data
    mbs_df = pd.read_csv(mbs_file)
    mbs_dict = {}
    for _, row in mbs_df.iterrows():
        mbs = MBS(
            issuer=row['issuer'],
            expiration_date=pd.to_datetime(row['expiration_date']).to_pydatetime(),
            coupon=float(row['coupon']),
            type=row['type'],
            amount=int(row['amount'] / 100000)  # Convert to $100k units
        )
        mbs_dict[mbs.id] = mbs
    
    # Load investor data
    investor_df = pd.read_csv(issuers_file)
    investors_dict = {}
    for _, row in investor_df.iterrows():
        allowed_issuers = eval(row['allowed_issuers']) if isinstance(row['allowed_issuers'], str) else row['allowed_issuers']
        investors_dict[row['investor']] = Investor(
            name=row['investor'],
            allowed_issuers=allowed_issuers,
            min_expiration_date=pd.to_datetime(row['min_expiration_date']).to_pydatetime(),
            min_allocation=int(row['min_allocation'] / 100000)  # Convert to $100k units
        )
    
    # Load loan data and aggregate by investor, note rate, and type
    loans_df = pd.read_csv(loans_file)
    grouped = loans_df.groupby(['investor', 'note_rate', 'note_type']).agg({'loan_amount': 'sum'}).reset_index()
    
    loan_pools_dict = {}
    for _, row in grouped.iterrows():
        pool = LoanPool(
            investor=row['investor'],
            note_rate=float(row['note_rate']),
            note_type=row['note_type'],
            total_amount=int(np.ceil(row['loan_amount'] / 100000))  # Convert to $100k units and round up
        )
        loan_pools_dict[pool.key] = pool
    
    return mbs_dict, investors_dict, loan_pools_dict

def build_eligibility_graph(mbs_dict: Dict[str, MBS], 
                           loan_pools_dict: Dict[str, LoanPool], 
                           investors_dict: Dict[str, Investor]) -> Dict[str, Set[str]]:
    """
    Build a bipartite graph linking MBS to eligible loan pools
    
    Returns:
        Dictionary mapping MBS IDs to sets of eligible loan pool keys
    """
    eligibility_graph = defaultdict(set)
    
    for mbs_id, mbs in mbs_dict.items():
        for pool_key, pool in loan_pools_dict.items():
            investor = investors_dict.get(pool.investor)
            if not investor:
                continue
                
            # Check eligibility criteria
            coupon_matches = abs(mbs.coupon - pool.note_rate) < 0.25  # Within 0.25% tolerance
            
            if (mbs.issuer in investor.allowed_issuers and
                mbs.expiration_date >= investor.min_expiration_date and
                mbs.type == pool.note_type and
                coupon_matches and
                mbs.amount >= investor.min_allocation):
                
                eligibility_graph[mbs_id].add(pool_key)
    
    return eligibility_graph

def maximum_bipartite_matching(eligibility_graph: Dict[str, Set[str]], 
                              mbs_dict: Dict[str, MBS],
                              loan_pools_dict: Dict[str, LoanPool],
                              investors_dict: Dict[str, Investor]) -> Allocation:
    """
    Greedy algorithm for maximum bipartite matching to allocate MBS to loan pools
    """
    # Initialize empty allocation
    allocation = Allocation(
        loan_pools_allocation={},
        mbs_allocation={}
    )
    
    # Track remaining amounts
    remaining_mbs = {mbs_id: mbs.amount for mbs_id, mbs in mbs_dict.items()}
    remaining_pools = {pool_key: pool.total_amount for pool_key, pool in loan_pools_dict.items()}
    
    # Initialize allocation tracking
    for pool_key in loan_pools_dict:
        allocation.loan_pools_allocation[pool_key] = 0
    
    # Create a priority queue based on "scarcity" of MBS
    # MBS that can fulfill fewer pools should be allocated first
    mbs_priority = [(len(eligible_pools), mbs_id) for mbs_id, eligible_pools in eligibility_graph.items()]
    mbs_priority.sort()  # Sort by number of eligible pools (ascending)
    
    # Allocate MBS in priority order
    for _, mbs_id in mbs_priority:
        if remaining_mbs[mbs_id] <= 0:
            continue
            
        mbs = mbs_dict[mbs_id]
        eligible_pools = eligibility_graph[mbs_id]
        
        # Sort pools by remaining amount (descending) to prioritize larger pools
        sorted_pools = [(remaining_pools[pool_key], pool_key) for pool_key in eligible_pools 
                        if remaining_pools[pool_key] > 0]
        sorted_pools.sort(reverse=True)
        
        if not sorted_pools:
            continue
            
        # Begin allocation
        allocation.mbs_allocation[mbs_id] = {}
        
        for _, pool_key in sorted_pools:
            pool = loan_pools_dict[pool_key]
            investor = investors_dict[pool.investor]
            min_allocation = investor.min_allocation
            
            # Skip if not enough MBS remaining for minimum allocation
            if remaining_mbs[mbs_id] < min_allocation:
                continue
                
            # Determine how much to allocate
            amount_to_allocate = min(
                remaining_mbs[mbs_id],  # Available MBS amount
                remaining_pools[pool_key]  # Remaining pool need
            )
            
            # Ensure minimum allocation size
            if amount_to_allocate >= min_allocation:
                # Update allocation
                allocation.mbs_allocation[mbs_id][pool_key] = amount_to_allocate
                allocation.loan_pools_allocation[pool_key] += amount_to_allocate
                allocation.total_allocated += amount_to_allocate
                
                # Update remaining amounts
                remaining_mbs[mbs_id] -= amount_to_allocate
                remaining_pools[pool_key] -= amount_to_allocate
                
                # Break if this MBS is fully allocated
                if remaining_mbs[mbs_id] < min_allocation:
                    break
    
    return allocation

def score_allocation(allocation: Allocation, loan_pools_dict: Dict[str, LoanPool]) -> float:
    """Calculate allocation score (percentage of total loans allocated)"""
    total_loan_amount = sum(pool.total_amount for pool in loan_pools_dict.values())
    if total_loan_amount == 0:
        return 0
    return (allocation.total_allocated / total_loan_amount) * 100

def refine_allocation_with_iterative_improvement(initial_allocation: Allocation,
                                              mbs_dict: Dict[str, MBS],
                                              loan_pools_dict: Dict[str, LoanPool],
                                              investors_dict: Dict[str, Investor],
                                              eligibility_graph: Dict[str, Set[str]]) -> Allocation:
    """
    Refine allocation using iterative improvement to increase total allocation
    
    This uses a simple local search to find improvements:
    1. Try reallocating smaller amounts from one pool to another
    2. Accept changes that increase total allocation
    """
    current_allocation = initial_allocation
    best_score = score_allocation(current_allocation, loan_pools_dict)
    
    # Clone the allocation for modification
    def clone_allocation(alloc):
        return Allocation(
            loan_pools_allocation=alloc.loan_pools_allocation.copy(),
            mbs_allocation={mbs_id: pools.copy() for mbs_id, pools in alloc.mbs_allocation.items()},
            total_allocated=alloc.total_allocated
        )
    
    improved = True
    iterations = 0
    max_iterations = 100  # Limit iterations to prevent infinite loops
    
    while improved and iterations < max_iterations:
        improved = False
        iterations += 1
        
        # Try reallocating MBS from one pool to another
        for mbs_id, pool_allocations in current_allocation.mbs_allocation.items():
            mbs = mbs_dict[mbs_id]
            eligible_pools = eligibility_graph[mbs_id]
            
            # For each pool already allocated this MBS
            for source_pool_key, allocated_amount in list(pool_allocations.items()):
                source_pool = loan_pools_dict[source_pool_key]
                source_investor = investors_dict[source_pool.investor]
                
                # For each pool that could use this MBS
                for target_pool_key in eligible_pools:
                    if target_pool_key == source_pool_key:
                        continue
                        
                    target_pool = loan_pools_dict[target_pool_key]
                    target_investor = investors_dict[target_pool.investor]
                    
                    # Calculate how much we can move
                    current_target_allocation = current_allocation.loan_pools_allocation.get(target_pool_key, 0)
                    remaining_target_need = target_pool.total_amount - current_target_allocation
                    
                    # Skip if target pool is already fulfilled or has no need
                    if remaining_target_need <= 0:
                        continue
                    
                    # Try moving allocation from source to target
                    for amount_to_move in range(target_investor.min_allocation, allocated_amount + 1):
                        # Skip if moving would violate minimum allocation for source
                        if allocated_amount - amount_to_move > 0 and allocated_amount - amount_to_move < source_investor.min_allocation:
                            continue
                            
                        # Skip if moved amount is greater than target need
                        if amount_to_move > remaining_target_need:
                            continue
                            
                        # Create a new allocation with this move
                        new_allocation = clone_allocation(current_allocation)
                        
                        # Remove allocation from source
                        if allocated_amount - amount_to_move >= source_investor.min_allocation:
                            new_allocation.mbs_allocation[mbs_id][source_pool_key] -= amount_to_move
                        else:
                            # Remove entire allocation if it would fall below minimum
                            new_allocation.mbs_allocation[mbs_id][source_pool_key] = 0
                            new_allocation.loan_pools_allocation[source_pool_key] -= allocated_amount
                            new_allocation.total_allocated -= allocated_amount
                        
                        # Add allocation to target
                        if target_pool_key in new_allocation.mbs_allocation[mbs_id]:
                            new_allocation.mbs_allocation[mbs_id][target_pool_key] += amount_to_move
                        else:
                            new_allocation.mbs_allocation[mbs_id][target_pool_key] = amount_to_move
                            
                        new_allocation.loan_pools_allocation[target_pool_key] += amount_to_move
                        new_allocation.total_allocated += amount_to_move
                        
                        # Calculate new score
                        new_score = score_allocation(new_allocation, loan_pools_dict)
                        
                        # Accept improvement
                        if new_score > best_score:
                            current_allocation = new_allocation
                            best_score = new_score
                            improved = True
                            break
                            
                    if improved:
                        break
                        
                if improved:
                    break
    
    print(f"Refinement completed in {iterations} iterations. New score: {best_score:.2f}%")
    return current_allocation


In [8]:
def generate_allocation_report(allocation: Allocation, 
                            mbs_dict: Dict[str, MBS], 
                            loan_pools_dict: Dict[str, LoanPool]) -> None:
    """Generate a comprehensive allocation report"""
    # Calculate fulfillment percentages
    fulfillment = {}
    for pool_key, pool in loan_pools_dict.items():
        allocated = allocation.loan_pools_allocation.get(pool_key, 0)
        percentage = (allocated / pool.total_amount * 100) if pool.total_amount > 0 else 0
        fulfillment[pool_key] = percentage
        
    # Categorize pools
    fully_allocated = []
    partially_allocated = []
    not_allocated = []
    
    for pool_key, pool in loan_pools_dict.items():
        percentage = fulfillment.get(pool_key, 0)
        allocated_amount = allocation.loan_pools_allocation.get(pool_key, 0)
        
        pool_info = {
            'pool': pool,
            'percentage': percentage,
            'allocated': allocated_amount * 100000,  # Convert back to dollars
            'allocations': []
        }
        
        # Collect MBS allocations for this pool
        for mbs_id, pool_allocations in allocation.mbs_allocation.items():
            if pool_key in pool_allocations and pool_allocations[pool_key] > 0:
                pool_info['allocations'].append({
                    'mbs': mbs_dict[mbs_id],
                    'amount': pool_allocations[pool_key] * 100000  # Convert back to dollars
                })
                
        # Categorize based on fulfillment percentage
        if percentage >= 99.9:
            fully_allocated.append(pool_info)
        elif percentage > 0:
            partially_allocated.append(pool_info)
        else:
            not_allocated.append(pool_info)
            
    # Sort each category by amount (descending)
    fully_allocated.sort(key=lambda x: x['allocated'], reverse=True)
    partially_allocated.sort(key=lambda x: x['allocated'], reverse=True)
    not_allocated.sort(key=lambda x: x['pool'].total_amount * 100000, reverse=True)
    
    # Print summary
    total_loan_amount = sum(pool.total_amount * 100000 for pool in loan_pools_dict.values())
    total_allocated_amount = allocation.total_allocated * 100000
    
    print("\n===== ALLOCATION SUMMARY =====")
    print(f"Total loan pools: {len(loan_pools_dict)}")
    print(f"Total amount: ${total_loan_amount:,.2f}")
    print(f"Amount allocated: ${total_allocated_amount:,.2f} ({total_allocated_amount/total_loan_amount*100:.1f}%)")
    print(f"Fully allocated pools: {len(fully_allocated)}")
    print(f"Partially allocated pools: {len(partially_allocated)}")
    print(f"Not allocated pools: {len(not_allocated)}")
    
    # Print details for each category
    print("\n===== FULLY ALLOCATED POOLS =====")
    for info in fully_allocated[:3]:  # Show first 3
        pool = info['pool']
        print(f"\n{pool.investor}: ${pool.total_amount * 100000:,.2f} of {pool.note_type} at {pool.note_rate}%")
        print(f"Allocation: ${info['allocated']:,.2f} ({info['percentage']:.1f}%)")
        for alloc in info['allocations']:
            mbs = alloc['mbs']
            print(f"  MBS: {mbs.issuer}, Exp: {mbs.expiration_date.strftime('%Y-%m-%d')}, Type: {mbs.type}, Amount: ${alloc['amount']:,.2f}")
            
    if partially_allocated:
        print("\n===== PARTIALLY ALLOCATED POOLS =====")
        for info in partially_allocated[:3]:
            pool = info['pool']
            print(f"\n{pool.investor}: ${pool.total_amount * 100000:,.2f} of {pool.note_type} at {pool.note_rate}%")
            print(f"Allocation: ${info['allocated']:,.2f} ({info['percentage']:.1f}%)")
            for alloc in info['allocations']:
                mbs = alloc['mbs']
                print(f"  MBS: {mbs.issuer}, Exp: {mbs.expiration_date.strftime('%Y-%m-%d')}, Type: {mbs.type}, Amount: ${alloc['amount']:,.2f}")
                
    if not_allocated:
        print("\n===== NOT ALLOCATED POOLS =====")
        for info in not_allocated[:3]:
            pool = info['pool']
            print(f"{pool.investor}: ${pool.total_amount * 100000:,.2f} of {pool.note_type} at {pool.note_rate}%")

## Execute Algorithm

In [9]:
def main():
    """Main function to run the global MBS allocation"""
    print("Loading data...")
    mbs_dict, investors_dict, loan_pools_dict = load_data(
        'ASC_815_short.csv',
        'allowed_issuers.csv',
        'loans.csv'
    )
    
    print(f"Loaded {len(mbs_dict)} MBS, {len(investors_dict)} investors, and {len(loan_pools_dict)} loan pools")
    
    # Build eligibility graph
    print("Building eligibility graph...")
    eligibility_graph = build_eligibility_graph(mbs_dict, loan_pools_dict, investors_dict)
    
    # Perform initial allocation
    print("Performing initial allocation...")
    initial_allocation = maximum_bipartite_matching(eligibility_graph, mbs_dict, loan_pools_dict, investors_dict)
    initial_score = score_allocation(initial_allocation, loan_pools_dict)
    print(f"Initial allocation score: {initial_score:.2f}%")
    
    # Refine allocation
    print("Refining allocation...")
    refined_allocation = refine_allocation_with_iterative_improvement(
        initial_allocation, mbs_dict, loan_pools_dict, investors_dict, eligibility_graph
    )
    
    # Generate report
    generate_allocation_report(refined_allocation, mbs_dict, loan_pools_dict)

In [10]:
if __name__ == "__main__":
    main()

Loading data...
Loaded 93 MBS, 6 investors, and 108 loan pools
Building eligibility graph...
Performing initial allocation...
Initial allocation score: 23.74%
Refining allocation...
Refinement completed in 14 iterations. New score: 28.83%

===== ALLOCATION SUMMARY =====
Total loan pools: 108
Total amount: $447,400,000.00
Amount allocated: $129,000,000.00 (28.8%)
Fully allocated pools: 5
Partially allocated pools: 60
Not allocated pools: 43

===== FULLY ALLOCATED POOLS =====

BankOfAmerica: $4,500,000.00 of GNMA II at 5.75%
Allocation: $4,500,000.00 (100.0%)
  MBS: C, Exp: 2025-03-15, Type: GNMA II, Amount: $1,000,000.00
  MBS: D, Exp: 2025-04-15, Type: GNMA II, Amount: $500,000.00
  MBS: E, Exp: 2025-03-15, Type: GNMA II, Amount: $1,000,000.00
  MBS: E, Exp: 2025-04-15, Type: GNMA II, Amount: $2,000,000.00

WellsFargo: $3,100,000.00 of GNMA II at 6.25%
Allocation: $3,100,000.00 (100.0%)
  MBS: F, Exp: 2025-04-15, Type: GNMA II, Amount: $1,500,000.00
  MBS: F, Exp: 2025-05-15, Type: GNM